In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

year=[] 
for i in range(1997,2018):
    year.append(i)

all_url=[]
for x in year:
   all_url.append('https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(x))

#Create function to scrape URL 
def urlScraping(url):
    
    nba=BeautifulSoup(urlopen(url))

    #Obtain Headers
    headers=[th.getText() for th in nba.findAll('tr')[0].findAll('th')]
    headers=headers[1:]

    #Obtain player data in rows
    rows = nba.findAll('tr')[1:]
    player_stats=[[td.getText() for td in rows[i].findAll('td')] for i in range (len(rows))]

    #Obtain playerID from rows
    player_id=[]
    player=[[data['data-append-csv'] for data in rows[i].findAll('td',{'data-append-csv':True})] for i in range(len(rows))]

    for x in player:
        for y in x:
            player_id.append(y)


    #Create stats data frame
    st=pd.DataFrame(player_stats, columns=headers)

    col= [c for c in st.columns if c.isspace()==False]

    st=st[col]

    st=st[st['Player'].notnull()]

    st.reset_index(inplace=True)

    #Add Year to DataFrame
    yr = [x.getText() for x in nba.findAll('span')]
    r=re.compile(r'\d')
    yr_list=list(filter(r.match, yr))
    nba_year=int(re.findall('\d+', yr_list[0])[0])
    nba_year=nba_year+1

    st['Year']=nba_year


    #Add PlayerID to stats
    st['player_id']=player_id

    #Drop index column
    st.drop('index', axis=1, inplace=True)

    #Convert objects to floats/int
    st[['G', 'MP', 'PER', 'TS%', '3PAr', 'FTr','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS','DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]=st[['G', 'MP', 'PER', 'TS%', '3PAr', 'FTr','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS','DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']].apply(pd.to_numeric)
    
    return st

In [7]:
#Loop to create CSV Files
#for url in all_url:
#    data = urlScraping(url)
#   name = 'data/nba_advancedstats_'+str(data['Year'][0])+'.csv'
#    data.to_csv(name)
    

In [8]:
#Concat nba_advanced_stats data
file_name='data/nba_advancedstats_{}.csv'
adv_stats_list=[]
for x in range(1997,2018):
    adv_stats_list.append(pd.read_csv(file_name.format(x)))

adv_stats=pd.concat(adv_stats_list)


In [73]:
#To get salary data, the URL needs to be setup

player_id=[x for x in adv_stats['player_id'].unique()]
first_letter = []
for x in player_id:
    y=x[0]
    first_letter.append(y)

tup=list(zip(first_letter,player_id))

salary_url=[]
for x in tup:
    salary_url.append('https://www.basketball-reference.com/players/{}/{}.html'.format(*x))
    
#Create a function
def salScraping(url):
    html=requests.get(url)
    html_doc=html.text.replace('<!--','').replace('-->','')
    content=BeautifulSoup(html_doc,'html.parser')
    sal=content.find(id='all_salaries')
    id=content.find('meta',attrs={'property':'og:url'})
    pl_id=re.findall(r'/www.basketball-reference.com/players/[a-z]/(.*).html', str(id))
    header=[th.getText() for th in sal.findAll('tr')[0].findAll('th')]
    header=header[1:]
    rows=sal.findAll('tr')[1:]
    team=sal.findAll('td', {'data-stat':'team_name'})
    pl_sal=[[td.getText() for td in rows[i].findAll('td')] for i in range (len(rows))]
    pl_season=[[th.getText() for th in rows[i].findAll('th')] for i in range (len(rows))]
    pl_team=[[x['href'] for x in team[i].findAll('a', {'href':True})] for i in range(len(team))]

    df_sal=pd.DataFrame(pl_sal,columns=header)
    season=[]
    for x in pl_season:
        for y in x:
           season.append(y) 
    teams=[]
    for x in pl_team:
        for y in x:
            y=re.findall(r'/teams/(.*)/', y)
            for z in y:
                teams.append(z)
    start_yr=[]
    for x in season:
        x=re.findall('\d\d\d\d',x)
        for y in x:
            start_yr.append(int(y))

    end_yr=[]
    for x in season:
        x=re.findall('\d\d\d\d',x)
        for y in x:
            y=int(y)+1
            end_yr.append(y)

    df_sal['Season']=season
    df_sal.drop(df_sal.index[-1], inplace=True)
    df_sal['team_id']=teams
    df_sal['season_start']=start_yr
    df_sal['season_end']=end_yr
    player_id=pd.DataFrame(pl_id, columns=['player_id'])
    df_salary=pd.concat([df_sal, player_id], sort=False)
    df_salary.bfill(inplace=True)
    df_salary.drop(df_salary.index[-1],inplace=True)
    df_salary.reset_index(inplace=True)
    df_salary.drop('index', axis=1, inplace=True)
    return df_salary

In [74]:
#Loop to create CSV Files
for url in salary_url:
    data = salScraping(url)
    name = 'data/nba_salary/nba_salary_'+str(data['player_id'][0])+'.csv'
    data.to_csv(name)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [4]:
#Salary and inflation
inflation=pd.read_csv('data/CPIAUCNS.csv', parse_dates=True)
salary=pd.read_csv('data/salaries_1985to2018.csv')

In [6]:
#Convert date to datetime in order to extract year
inflation.DATE=pd.to_datetime(inflation.DATE)
inflation['YEAR']=inflation.DATE.dt.year

#Narrow down inflation to Year and Inflation
inflation=inflation[['YEAR','CPIAUCNS']]

#Create a mean for each year
inflation=inflation.groupby('YEAR').mean()

#Reset Index
inflation=inflation.reset_index()

#Create a multiplier for inflation
inflation['CPIMult']= (inflation.iloc[-1,1]) / (inflation['CPIAUCNS']) 

In [9]:
#Merge salary and inflation to adjust salary
adj_salary=salary.merge(inflation, left_on='season_end', right_on='YEAR')

#Multiply salary by inflation multiplier to get adjusted salary
adj_salary['adj_salary']=(adj_salary.salary*adj_salary.CPIMult).round()

#Display on certain fields
adj_salary=adj_salary[['player_id', 'adj_salary','YEAR', 'season', 'team']]

In [10]:
#Merge adv_stats and adj_salary

df=pd.merge(adv_stats,adj_salary,how='left',left_on=['player_id','Year'], right_on=['player_id','YEAR'])

#Removing null values on adj_salary because it accounted for less than 5% of the data
df=df[df.adj_salary.notnull()]

In [76]:
#print (df)
